In [135]:
import pandas as pd
import numpy as np

In [136]:
df=pd.read_csv("train1.csv")

In [137]:
df.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [138]:
test=pd.read_csv("test1.csv")

In [139]:
df['No. of topics'] = (df['Computer Science'] + df['Physics'] + df['Mathematics'] + 
                       df['Statistics'] + df['Quantitative Biology'] + df['Quantitative Finance'])

df.head(3)

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,No. of topics
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0,1
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0,1
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0,1


In [140]:
def data_clean(df):
    df['total'] = df['TITLE'] + df['ABSTRACT']
    df = df.drop(labels=['ID', 'No. of topics', 'TITLE', 'ABSTRACT'], axis=1, inplace=True)
    return df

In [141]:
data_clean(df)
df.head(3)

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,total
0,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps P...
1,1,0,0,0,0,0,Rotation Invariance Neural Network Rotation i...
2,0,0,1,0,0,0,Spherical polyharmonics and Poisson kernels fo...


In [142]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.multiclass import OneVsRestClassifier

In [143]:
tfidf = TfidfVectorizer(analyzer='word', max_features=5000, ngram_range=(1,3), stop_words='english')
X = tfidf.fit_transform(df['total'])

In [144]:
X.shape

(20972, 5000)

In [145]:
df.columns

Index(['Computer Science', 'Physics', 'Mathematics', 'Statistics',
       'Quantitative Biology', 'Quantitative Finance', 'total'],
      dtype='object')

In [146]:
y=df[['Computer Science', 'Physics', 'Mathematics', 'Statistics',
       'Quantitative Biology', 'Quantitative Finance']]

In [147]:
y.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,1,0,0,1,0,0


In [148]:
y.shape

(20972, 6)

In [149]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [150]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(SGDClassifier(), n_jobs=-1)),
            ])
for y in y:
    print('**Processing {} comments...**'.format(y))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(X_train, y_train)
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))
    print("\n")

**Processing Computer Science comments...**
Test accuracy is 0.6600715137067938


**Processing Physics comments...**
Test accuracy is 0.6619785458879619


**Processing Mathematics comments...**
Test accuracy is 0.6600715137067938


**Processing Statistics comments...**
Test accuracy is 0.6603098927294399


**Processing Quantitative Biology comments...**
Test accuracy is 0.6584028605482718


**Processing Quantitative Finance comments...**
Test accuracy is 0.6595947556615018




In [151]:
a=pd.DataFrame(prediction)

In [152]:
a

,0,1,2,3,4,5
0,0,0,1,0,0,0
1,0,0,1,0,0,0
2,0,1,0,0,0,0
3,0,0,1,0,0,0
4,0,0,1,0,0,0
...,...,...,...,...,...,...
4190,0,0,1,0,0,0
4191,0,1,0,0,0,0
4192,0,0,1,0,0,0
4193,0,1,0,0,0,0


In [153]:
test=pd.read_csv("test1.csv")

In [154]:
test.head()

,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...


In [155]:
def data_clean_test(test):
    test['total'] = test['TITLE'] + test['ABSTRACT']
    test = test.drop(labels=['ID','TITLE', 'ABSTRACT'], axis=1, inplace=True)
    return test

In [156]:
data_clean_test(test)
test.head(3)

,total
0,Closed-form Marginal Likelihood in Gamma-Poiss...
1,Laboratory mid-IR spectra of equilibrated and ...
2,Case For Static AMSDU Aggregation in WLANs Fr...


In [157]:
z = tfidf.transform(test['total'])

In [158]:
z

<8989x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 538490 stored elements in Compressed Sparse Row format>

In [159]:
prediction1 = LogReg_pipeline.predict(z)

In [160]:
prediction1.shape

(8989, 6)

In [196]:
sub=pd.read_csv("sub.csv")

In [197]:
sub.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,0,0,0
1,20974,0,0,0,0,0,0
2,20975,0,0,0,0,0,0
3,20976,0,0,0,0,0,0
4,20977,0,0,0,0,0,0


In [198]:
sub1=pd.DataFrame(prediction1)

In [199]:
sub1

,0,1,2,3,4,5
0,0,0,0,1,0,0
1,0,1,0,0,0,0
2,1,0,0,0,0,0
3,0,1,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
8984,1,0,0,0,0,0
8985,0,0,1,0,0,0
8986,0,0,0,0,1,0
8987,0,0,0,1,0,0


In [166]:
sub1.head(20)

,0,1,2,3,4,5
0,0,0,0,1,0,0
1,0,1,0,0,0,0
2,1,0,0,0,0,0
3,0,1,0,0,0,0
4,1,0,0,0,0,0
5,0,0,0,1,0,0
6,0,0,1,1,0,0
7,1,1,0,0,0,0
8,1,0,0,0,0,0
9,0,1,0,0,0,0


In [167]:
sub1.to_csv('result.csv')

In [200]:
sub.columns

Index(['ID', 'Computer Science', 'Physics', 'Mathematics', 'Statistics',
       'Quantitative Biology', 'Quantitative Finance'],
      dtype='object')

In [201]:
sub['Computer Science']=sub1[0]

In [202]:
sub['Physics']=sub1[1]
sub['Mathematics']=sub1[2]
sub['Statistics']=sub1[3]
sub['Quantitative Biology']=sub1[4]
sub['Quantitative Finance']=sub1[5]

In [203]:
sub.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0


In [204]:
sub.to_csv('submission_final.csv',index=False)

In [194]:
sub.shape

(8989, 9)

In [183]:
k=pd.read_csv("test1.csv")

In [184]:
k.shape

(8989, 3)

In [185]:
k.head()

,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...


In [186]:
sub['TITLE']=k['TITLE']

In [187]:
sub['ABSTRACT']=k['ABSTRACT']

In [188]:
sub.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,TITLE,ABSTRACT
0,20973,0,0,0,1,0,0,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,0,1,0,0,0,0,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,1,0,0,0,0,0,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,0,1,0,0,0,0,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,1,0,0,0,0,0,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...


In [189]:
sub.columns

Index(['ID', 'Computer Science', 'Physics', 'Mathematics', 'Statistics',
       'Quantitative Biology', 'Quantitative Finance', 'TITLE', 'ABSTRACT'],
      dtype='object')

In [190]:
final1=sub[['ID', 'TITLE', 'ABSTRACT', 'Computer Science', 'Physics', 'Mathematics', 'Statistics',
       'Quantitative Biology', 'Quantitative Finance']]

In [191]:
final1.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...,0,0,0,1,0,0
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...,0,1,0,0,0,0
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...,1,0,0,0,0,0
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...,0,1,0,0,0,0
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...,1,0,0,0,0,0


In [192]:
final1.to_csv('submission_final1.csv')